# Evaluating Senti-wordnet

In this part of our experiments we are evaluating different strategies for SentiWordNet

In [56]:
import pandas as pd
pd.set_option('display.max_colwidth', 1200)

from Politweet import get_tweets

from simpleSentiWordNetClassifier import simpleSentiWordNetClasifier as swn_simple
from AdjBasedSentiWordNet import adjBasedSentiWordNetClassifier as swn_adj
from AvgBasedSentiWordNetClassifier import avgBasedClassifier as swn_avg
from SummationBasedClassifier import sumBasedClassifier as swn_sum
from nltk.corpus import sentiwordnet as swn
import ratings
import Politweet as ptweet
import nltk
from sentiment import minus_df, plus_df

def accuracy(results, gold):
    correct = [g == r for (g, r) in zip(gold, results)]
    return float(sum(correct))/len(correct)

tweets = ptweet.get_tweets('./datasets/tweets.tsv');

### The test set
In order to test the accuracy of different implementations, we are going to use only the tweets whose ATM rating is coherent.

In [73]:
pos = ratings.all(tweets, ratings.POSITIVE)
pos = ptweet.df_setminus(pos, plus_df(pos))

neg = ratings.all(tweets, ratings.NEGATIVE)
neg = ptweet.df_setminus(neg, minus_df(neg))
print "Total", len(pos) + len(neg)
print "Positive", len(pos)
print "Negative", len(neg)

Total 658
Positive 198
Negative 460


In [74]:
import sentlex
SWN = sentlex.SWN3Lexicon()
swn_classifier = sentlex.sentanalysis.BasicDocSentiScore()

def swn_classify_s(text):
    return classifier.classify_document(
        text,
        tagged=False,
        L=SWN,
        a=True,
        v=True,
        n=True,
        r=True,
        negation=True,
        verbose=False)

### Running SWN

In [75]:
def prob_to_score(x):
    pos = x[0]
    neg = x[1]
    if pos == neg:
        return ratings.MIXED
    elif pos > neg:
        return ratings.POSITIVE
    else:
        return ratings.NEGATIVE

def run_swn(t, swn_f=swn_simple):
    data = t.copy(deep=True)
    data['simple'] = data['content'].apply(swn_simple)
    data['adj'] = data['content'].apply(swn_adj)
    data['avg'] = data['content'].apply(swn_avg)
    data['sum'] = data['content'].apply(swn_sum)
    data['sentlex_s'] = data['content'].apply(swn_classify_s)
    data['sentlex'] = data['sentlex_s'].apply(prob_to_score)
    return data

scored_pos = run_swn(pos)
scored_neg = run_swn(neg)

In [76]:
scored_pos.to_pickle('./datasets/swn_pos.pickle')
scored_neg.to_pickle('./datasets/swn_neg.pickle')

### Scoring accuracy

In [79]:
def score_accuracy(data):
    print "simple", accuracy(data['simple'], data['rating.1'])
    print "adj", accuracy(data['adj'], data['rating.1'])
    print "sum", accuracy(data['sum'], data['rating.1'])
    print "avg", accuracy(data['avg'], data['rating.1'])
    print "sentlex", accuracy(data['sentlex'], data['rating.1'])

print "Positive:"
score_accuracy(scored_pos)
print "\nNegative:"
score_accuracy(scored_neg)

scored_neg[['content', 'sentlex_s', 'sentlex']]

Positive:
simple 0.343434343434
adj 0.343434343434
sum 0.171717171717
avg 0.681818181818
sentlex 0.757575757576

Negative:
simple 0.323913043478
adj 0.25652173913
sum 0.289130434783
avg 0.371739130435
sentlex 0.445652173913


,content,sentlex_s,sentlex
tweet.id,,,
936470432,"@ahg3 @MichDot Yeah, slime was actually my second choice, can't say what the first one was. Okay, we're rolling...","(0.312111499611, 0.302083333333)",2
936472030,Preparing to have a heart attack #tweetdebate,"(0.142160087719, 0.125274122807)",2
936472928,Introducing McCain and Obama. Wait...McCain SHOWED UP! Wow...guess the crisis in Washington is over? #tweetdebate,"(0.0288461538462, 0.4375)",1
936479031,#tweetdebate McCain starts awkwardly,"(0.125, 0.0267857142857)",2
936480659,@current McCain looks so tired!! wayyyy past his bedtime #current,"(0.09375, 0.0)",2
936481174,@andrewa121 He looks and sounds pretty tired.,"(0.135416666667, 0.258333333333)",1
936482946,"Nice, McLame divides...Republicans uniquely came to fix it. Um, ok. #tweetdebate","(0.0238095238095, 0.0540674603175)",1
936482963,"if ur oldest candidate in history, why start with ""i'm not feeling great"" .. makes u thing ""you're not looking great either #tweetdebate","(0.765269328885, 1.34316060215)",1
936490363,how annoying are these animations on #Current TV.. reverting to old fashion MSNBC.. #tweetdebate,"(0.291346153846, 0.015625)",2
